In [1]:
%run generate_matrices_tools.ipynb     # importing functions

# Rounding detection

This notebook's purpose is to test a following hypothesis: given a "cube" defined by real-valued coordinates from the cube algorithm, assume a rounding of these coordinates, which are in some sense *nearly* an integer. We test whether this rounding gives us the desired coordinate of the linear combination given by the LLL. 

Note: this may not work always, as sometimes the jump is greater than 1.

In [19]:
def detect_rounding(jsonfilename):
    '''
    OUTPUT: dict in the form \{ limit: [succesful attempts, unsuc att.],...\}
    '''
    cases = from_json(jsonfilename)
    stats = {}
    for limit in limits:
        stats[limit] = [0, 0, 0, 0] 
    for case in cases:
        for i in range(len(case["lincomb_cube"])):
            
            #skip equal components
            if case["lincomb_diff"][i] == 0:
                continue
                
            for limit in limits:
                num_cube = case["lincomb_cube"][i]
                num_LLL = case["lincomb_LLL"][i]
                is_near_int, direction =  is_nearest_integer(num_cube, limit)
                if is_near_int: # if its almost an int
                    if round(num_cube) == num_LLL: 
                        stats[limit] = increment(stats, limit, 0)                        # rounding lead directly to lcLLL
                    else:
                        print(num_LLL, "\t",num_cube,"\t", lcLLL_is_in_same_direction(direction, num_cube, num_LLL),"\t",case["lincomb_cube"])
                        print("\t","\t","\t","\t","\t","",case["lincomb_LLL"])
                        stats[limit] = increment(stats, limit, 1)                        # rounding didnt lead directly to lcLLL
                        if lcLLL_is_in_same_direction(direction, num_cube, num_LLL):
                            stats[limit] = increment(stats, limit, 2)                    # but it was in the same direction
                        else: stats[limit] = increment(stats, limit, 3)
    return stats

def is_nearest_integer(num, tolerance):
    nearest_int = round(num)
    is_within_tolerance = abs(num - nearest_int) <= tolerance
    if not is_within_tolerance:
        return False, None
    direction = "up" if num - nearest_int < 0 else "down"
    return True, direction


def increment(dic, limit, ind):
    lst = [dic.get(limit,0)[0], dic.get(limit,0)[1],  dic.get(limit,0)[2],  dic.get(limit,0)[3]]
    lst[ind] += 1
    return lst

def lcLLL_is_in_same_direction(direction, num_cube, num_LLL, ):
    return abs(num_cube - num_LLL) - abs(round(num_cube) - num_LLL) > 0
